# Data Science Nigeria Staff Promotion Compitition
### Predicting staff that are likely to be promoted based on defined personal and performance parameters

In [56]:
import os
for dirname, _, filenames in os.walk('C:/Users/Arthur/data-projects/DataScienceNigeria_ML_Course_Practice/'):
#for dirname, _, filenames in os.walk('C:\\Users\\Arthur\\data-projects\\DataScienceNigeria_ML_Course_Practice'):   
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:/Users/Arthur/data-projects/DataScienceNigeria_ML_Course_Practice/ds_nigeria_ml_course_notebook.ipynb
C:/Users/Arthur/data-projects/DataScienceNigeria_ML_Course_Practice/.ipynb_checkpoints\ds_nigeria_ml_course_notebook-checkpoint.ipynb
C:/Users/Arthur/data-projects/DataScienceNigeria_ML_Course_Practice/datasets\sample_submission2.csv
C:/Users/Arthur/data-projects/DataScienceNigeria_ML_Course_Practice/datasets\test.csv
C:/Users/Arthur/data-projects/DataScienceNigeria_ML_Course_Practice/datasets\train.csv


In [57]:
# Graphing imports
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [112]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, f1_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV, KFold
from sklearn.linear_model import Ridge, RidgeCV, LogisticRegressionCV, LogisticRegression
#from sklearn.svm import LinearSVR
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.compose import ColumnTransformer

In [59]:
from scipy.stats import uniform, randint
import xgboost as xgb

# EDA

In [117]:
data = pd.read_csv('C:/Users/Arthur/data-projects/DataScienceNigeria_ML_Course_Practice/datasets/train.csv')
data.head()

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
2,YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
3,YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
4,YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38312 entries, 0 to 38311
Data columns (total 19 columns):
EmployeeNo                             38312 non-null object
Division                               38312 non-null object
Qualification                          36633 non-null object
Gender                                 38312 non-null object
Channel_of_Recruitment                 38312 non-null object
Trainings_Attended                     38312 non-null int64
Year_of_birth                          38312 non-null int64
Last_performance_score                 38312 non-null float64
Year_of_recruitment                    38312 non-null int64
Targets_met                            38312 non-null int64
Previous_Award                         38312 non-null int64
Training_score_average                 38312 non-null int64
State_Of_Origin                        38312 non-null object
Foreign_schooled                       38312 non-null object
Marital_Status                         383

In [33]:
print(data.columns)
print(data.No_of_previous_employers.unique())
print(data.Previous_IntraDepartmental_Movement.unique())
print(data.Past_Disciplinary_Action.unique())
print(data.Marital_Status.unique())
print(data.Foreign_schooled.unique())
print(data.State_Of_Origin.unique())
print(data.Year_of_recruitment.unique())
print(data.Channel_of_Recruitment.unique())
print(data.Year_of_birth.unique())
print(data.Gender.unique())
print(data.Qualification.unique())
print(data.Division.unique())

Index(['EmployeeNo', 'Division', 'Qualification', 'Gender',
       'Channel_of_Recruitment', 'Trainings_Attended', 'Year_of_birth',
       'Last_performance_score', 'Year_of_recruitment', 'Targets_met',
       'Previous_Award', 'Training_score_average', 'State_Of_Origin',
       'Foreign_schooled', 'Marital_Status', 'Past_Disciplinary_Action',
       'Previous_IntraDepartmental_Movement', 'No_of_previous_employers',
       'Promoted_or_Not'],
      dtype='object')
['0' '1' '4' '3' '2' 'More than 5' '5']
['No' 'Yes']
['No' 'Yes']
['Married' 'Single' 'Not_Sure']
['No' 'Yes']
['ANAMBRA' 'KATSINA' 'NIGER' 'AKWA IBOM' 'ENUGU' 'OYO' 'LAGOS' 'ONDO'
 'KWARA' 'EDO' 'BAUCHI' 'TARABA' 'RIVERS' 'KADUNA' 'PLATEAU' 'EKITI'
 'BORNO' 'IMO' 'CROSS RIVER' 'KANO' 'FCT' 'OGUN' 'GOMBE' 'DELTA' 'BENUE'
 'ZAMFARA' 'OSUN' 'ABIA' 'KEBBI' 'ADAMAWA' 'NASSARAWA' 'SOKOTO' 'KOGI'
 'JIGAWA' 'BAYELSA' 'YOBE' 'EBONYI']
[2011 2015 2012 2009 2014 2013 2016 2003 2010 2008 2018 2017 2007 2002
 2005 2006 2000 2004 2001 199

In [34]:
print(data.isnull().values.any())
print(data.shape)

True
(38312, 19)


In [95]:
data.isnull().sum()

EmployeeNo                                0
Division                                  0
Qualification                          1679
Gender                                    0
Channel_of_Recruitment                    0
Trainings_Attended                        0
Year_of_birth                             0
Last_performance_score                    0
Year_of_recruitment                       0
Targets_met                               0
Previous_Award                            0
Training_score_average                    0
State_Of_Origin                           0
Foreign_schooled                          0
Marital_Status                            0
Past_Disciplinary_Action                  0
Previous_IntraDepartmental_Movement       0
No_of_previous_employers                  0
Promoted_or_Not                           0
dtype: int64

In [118]:
X = data
y = data.Promoted_or_Not
z = data.EmployeeNo
print(X.shape, y.shape, z.shape)
X = X.drop(labels=['Promoted_or_Not', 'EmployeeNo'], axis=1) # dropped EmployeeNo since I cant categorize it
print(X.shape, y.shape, z.shape)

(38312, 19) (38312,) (38312,)
(38312, 17) (38312,) (38312,)


In [36]:
#import winsound
#winsound.Beep(4300, 1000)

In [37]:
X.head()    

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers
0,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0
1,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0
2,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0
3,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1
4,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1


In [62]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38312 entries, 0 to 38311
Data columns (total 17 columns):
Division                               38312 non-null object
Qualification                          36633 non-null object
Gender                                 38312 non-null object
Channel_of_Recruitment                 38312 non-null object
Trainings_Attended                     38312 non-null int64
Year_of_birth                          38312 non-null int64
Last_performance_score                 38312 non-null float64
Year_of_recruitment                    38312 non-null int64
Targets_met                            38312 non-null int64
Previous_Award                         38312 non-null int64
Training_score_average                 38312 non-null int64
State_Of_Origin                        38312 non-null object
Foreign_schooled                       38312 non-null object
Marital_Status                         38312 non-null object
Past_Disciplinary_Action               383

In [63]:
print(X.isnull().values.any())
print(y.isnull().values.any())
print(z.isnull().values.any())

True
False
False


In [78]:
imp = SimpleImputer(strategy='most_frequent')
#X_imputed = imp.fit_transform(X)
#X_imputed

In [75]:
#X_imputed = pd.DataFrame.from_records(X_imputed)
#X_imputed.columns = X.columns
#print(X_imputed.columns is X.columns)

In [76]:
print(X.isnull().values.any())
print(X_imputed.isnull().values.any())
print(y.isnull().values.any())
print(z.isnull().values.any())

True
False
False
False


In [119]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]
print(numerical_cols)
categorical_cols = [cname for cname in X.columns if X[cname].dtype == 'object']
print(categorical_cols)
print(len(numerical_cols) + len(categorical_cols))

['Trainings_Attended', 'Year_of_birth', 'Last_performance_score', 'Year_of_recruitment', 'Targets_met', 'Previous_Award', 'Training_score_average']
['Division', 'Qualification', 'Gender', 'Channel_of_Recruitment', 'State_Of_Origin', 'Foreign_schooled', 'Marital_Status', 'Past_Disciplinary_Action', 'Previous_IntraDepartmental_Movement', 'No_of_previous_employers']
17


In [120]:
X_train = imp.fit_transform(X_train)
X_test = imp.transform(X_test)

In [121]:
X_train = pd.DataFrame.from_records(X_train)
X_train.columns = X.columns
print(X_train.columns is X.columns)

X_test = pd.DataFrame.from_records(X_test)
X_test.columns = X.columns
print(X_test.columns is X.columns)

print(X_train.isnull().values.any())
print(X_test.isnull().values.any())

True
True
False
False


In [122]:
X_test.head()

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers
0,Sourcing and Purchasing,"MSc, MBA and PhD",Male,Agency and others,2,1986,12.5,2010,1,0,63,LAGOS,Yes,Married,No,No,0
1,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,2,1968,12.5,2010,1,0,44,OYO,Yes,Single,No,No,0
2,Customer Support and Field Operations,"MSc, MBA and PhD",Male,Agency and others,2,1983,12.5,2012,1,0,46,RIVERS,Yes,Married,No,No,1
3,Customer Support and Field Operations,"MSc, MBA and PhD",Male,Agency and others,2,1974,12.5,2013,1,0,52,OSUN,Yes,Single,No,No,1
4,Information Technology and Solution Support,First Degree or HND,Female,Agency and others,2,1975,7.5,2000,0,0,71,LAGOS,Yes,Married,No,No,5


In [123]:
#Transform nan values using SimpleImputer -- a
#Transform numerical cols using StandardScaler -- b
#Transform categorical cols using OneHotEncoder -- c  
#[steps b and c can be done using FeatureUnion or ColumnTransformer]

#Need to do 
#Cross validation on the dataset -- d
#Hyperparameter tune and search best one -- e
#Use early stopping if model doesn't iteratively improve -- f

# Set params_grid for hyper-parameter tuning
params = {"colsample_bytree": uniform(0.7, 0.3),
          "gamma": uniform(0, 0.5),
          "learning_rate": uniform(0.03, 0.3), # default 0.1 
          "max_depth": randint(1, 7), # default 3
          "n_estimators": randint(50, 150), # default 100
          "subsample": uniform(0.6, 0.4)
         }

# Search through the params_grid
#search_params_model = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, 
#                            n_iter=200, cv=3, verbose=1, n_jobs=1, return_train_score=True)

# Use early stopping if model doesn't improve
#xgb_model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_test, y_test)])

#xgb_model.fit() will produce a model from the last iteration, not the best one, 
#so to get the optimum model consider retraining over xgb_model.best_iteration rounds.

l_model = xgb.XGBClassifier()
#l_model = GradientBoostingClassifier()
scaler = StandardScaler()
#imp = SimpleImputer(strategy='most_frequent')
ohe = OneHotEncoder(handle_unknown='ignore')
col_transformer = ColumnTransformer([('scaler', scaler, numerical_cols), 
                                     #('imp', imp, ['Qualification']),
                                     ('ohe', ohe, categorical_cols)],
                                    remainder='passthrough')
pipe = Pipeline([('col_transformer', col_transformer),
                 ('model', l_model)],
                verbose=1)


In [124]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

[Pipeline] ... (step 1 of 2) Processing col_transformer, total=   0.6s
[Pipeline] ............. (step 2 of 2) Processing model, total=   8.3s


In [125]:
#XGB scores -- 0.5090
print(accuracy_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.9433642176693201
0.05663578233067989
0.5090497737556562
[[7004    3]
 [ 431  225]]


In [87]:
#GBC scores -- 0.5179
print(accuracy_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.9438862064465614
0.0561137935534386
0.5179372197309416
[[7002    5]
 [ 425  231]]


In [115]:
#ColumnTransformer?
#FeatureUnion?
#Pipeline?
#StandardScaler?
#OneHotEncoder?
#SimpleImputer?
#LogisticRegression?
#LogisticRegressionCV?
#train_test_split?
#GridSearchCV?
#RandomizedSearchCV?
#KFold?
#Ridge?

In [ ]:
test_data = pd.read_csv('C:/Users/Arthur/data-projects/DataScienceNigeria_ML_Course_Practice/datasets/train.csv')
test_data.head()

In [ ]:
test_data.info()

In [ ]:
print(test_data.isnull().values.any())
print(test_data.shape)

In [ ]:
test_X = test_data
#y = data.Promoted_or_Not
test_z = test_data.EmployeeNo
print(test_X.shape, test_z.shape)
test_X = test_X.drop(labels='EmployeeNo', axis=1) # dropped EmployeeNo since I cant categorize it
print(test_X.shape, test_z.shape)

In [ ]:
#imp = SimpleImputer(strategy='most_frequent')
#X_imputed = imp.fit_transform(X)
test_X_imputed = imp.transform(test_X)
test_X_imputed = pd.DataFrame.from_records(test_X_imputed, columns=test_X.columns)
#test_X_imputed.columns = test_X.columns
print(test_X_imputed.columns is test_X.columns)

In [ ]:
print(test_X_imputed.isnull().values.any())
print(test_X_imputed.shape, X_imputed.shape)

In [ ]:
test_X_imputed.head()

In [ ]:
test_pred = pipe.predict(test_X_imputed)
test_pred

In [ ]:
output = pd.DataFrame({'EmployeeNo': test_z, 'Promoted_or_Not': test_pred})
output.to_csv('submission_1.csv', index=False)

In [ ]:
#Test different classifiers to find the best performer out of the box with default values
clfs = {#'mnb': MultinomialNB(),  #found -ve values....
        'gnb': GaussianNB(),
        'ber': BernoulliNB(),
        'svm1': SVC(kernel='linear'),
        'svm2': SVC(kernel='rbf'),
        'svm3': SVC(kernel='sigmoid'),
        'mlp1': MLPClassifier(),
        'mlp2': MLPClassifier(hidden_layer_sizes=[100, 100]),
        'ada': AdaBoostClassifier(),
        'dtc': DecisionTreeClassifier(),
        'rfc': RandomForestClassifier(),
        'gbc': GradientBoostingClassifier(),   #Seems like this performs best
        'lr': LogisticRegression(),
        'xgb': xgb.XGBClassifier()             #Seems like this performs second best
       }


f1_scores = dict()
for clf_name in clfs:
    clf = clfs[clf_name]
    pipe = Pipeline([('col_transformer', col_transformer),
                 ('model', clf)],
                verbose=0)    #do not want verbose output
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    f1_scores[clf_name] = f1_score(y_test, y_pred)
    
f1_scores

In [154]:
#gbc = GradientBoostingClassifier(),
#xgb = xgb.XGBClassifier()
estimators = {'gbc': GradientBoostingClassifier(),
              #'xgb': xgb.XGBClassifier()
             }

# Set params_grid for xgb hyper-parameter tuning
xgb_params = {'max_depth': randint(1, 7),          # default 3
              'learning_rate': uniform(0.03, 0.5), # default 0.1
              'n_estimators': randint(50, 150),    # default 100
              'gamma': uniform(0, 1.5),            # default 0
              'min_child_weight': randint(0, 3),   # default 1
              'max_delta_step': randint(0, 3),     # default 0
              'subsample': uniform(0.6, 0.4),      # default 1 **
              'colsample_bytree': uniform(0.7, 0.3),   # default 1
              'colsample_bylevel': uniform(0.7, 0.3),  # default 1
              'colsample_bynode': uniform(0.7, 0.3),   # default 1
              'reg_alpha': uniform(0, 1.5),            # default 0
              'reg_lambda': uniform(0.6, 0.4),         # default 1
              'scale_pos_weight': uniform(0.7, 0.3),   # default 1
              'base_score': uniform(0.7, 0.3),         # default 0.5   
             }

# Set params_grid for gbc hyper-parameter tuning
gbc_params = {'loss': ['deviance', 'exponential'],     # default deviance
              #'learning_rate': uniform(0.03, 0.5),     # default 0.1
              'learning_rate': np.logspace(-0.9, , 10),     # default 0.1
              #'n_estimators': randint(50, 150),        # default 100
              'n_estimators': range(50, 150, 5),        # default 100
              'subsample': np.logspace(0.5, 1.5, 10),          # default 1
              'criterion': ['friedman_mse', 'mse', 'mae'],   # default friedman_mse
              'min_samples_split': range(1, 7),            # default 2
              'min_samples_leaf': range(1, 7),             # default 1
              'min_weight_fraction_leaf': np.logspace(0.01, 1.59, 10),     # default 0.0
              'max_depth': range(1, 7),                          # default 3,
              'min_impurity_decrease': np.logspace(0.01, 1.59, 10),        # default 0.0
              'warm_start': [False, True],                         # default False
              'validation_fraction': np.logspace(0.01, 1.00, 10),          # default 0.1,
              'n_iter_no_change': range(1, 7)                    # default None,
             }

hyper_param_dict = {'gbc': gbc_params, 
                    #'xgb': xgb_params
                   }
scores = {}
#f1_scores = {}

# Use this to get the best hyperparams for the two estimators
# Then use one of them in the final model with a search grid
for name, estimator in estimators.items():
    search_model = GridSearchCV(estimator, 
                                hyper_param_dict[name], 
                                cv=5, 
                                scoring='f1', 
                                verbose=1)
    search_model.fit(X_train, y_train)
    scores[name] = -search_model.best_score_
    #f1_scores[name] = f1_score(y_test, y_pred)
    
print(scores)    
#print(f1_scores)

MemoryError: 

In [160]:
print(np.logspace(0.01, 1.00))
print(len(np.logspace(0.01, 1.00)))

[ 1.02329299  1.07202295  1.12307346  1.17655503  1.23258343  1.29127995
  1.35277164  1.41719161  1.4846793   1.5553808   1.62944917  1.70704472
  1.78833543  1.87349726  1.96271455  2.05618044  2.15409725  2.25667692
  2.36414151  2.47672365  2.59466703  2.71822695  2.84767089  2.98327904
  3.12534496  3.27417615  3.43009479  3.5934384   3.76456054  3.94383164
  4.13163977  4.32839145  4.53451259  4.75044937  4.97666922  5.21366181
  5.46194017  5.72204173  5.9945295   6.27999335  6.57905119  6.89235038
  7.2205691   7.56441784  7.92464091  8.30201805  8.69736618  9.11154106
  9.54543926 10.        ]
50


In [144]:
#xgb.XGBClassifier?
GradientBoostingClassifier?
#randint?
#uniform?

In [130]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

import xgboost as xgb

C:\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [17]:
GradientBoostingClassifier?
#RandomizedSearchCV?
#np.linspace(0.1, 1.0, 10, endpoint=True)
#GradientBoostingClassifier.fit?
#xgb.XGBClassifier.fit?